In [1]:
import pandas as pd

# 读取jsonl文件
file_path = 'comment_20240626225604.jsonl'
df = pd.read_json(file_path, lines=True)

# 提取content字段
contents = df['content'].tolist()

In [2]:
len(contents)

6282

In [3]:
from transformers import BertTokenizer, BertModel
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

d:\Anaconda\envs\gpaTorch\lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Anaconda\envs\gpaTorch\lib\site-packages\transformers\utils\generic.py:339: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
MODEL_PATH = "hfl/Chinese-bert-wwm"

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)   # 分词器
bert = BertModel.from_pretrained(MODEL_PATH)            # 模型

In [5]:
input_size = 768

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        out = self.sigmoid(out)
        return out

net = Net(input_size).to(device)

In [7]:
net = torch.load("bert_dnn_8.model")

In [41]:
# s = ["我爱你们"]
outputs_list = []
for batch in range(629):
    if (batch != 628):
        contents_1 = contents[batch*10:batch*10+10]
        tokens = tokenizer(contents_1, padding=True)
        input_ids = torch.tensor(tokens["input_ids"])
        attention_mask = torch.tensor(tokens["attention_mask"])
        with torch.no_grad():  # 禁用梯度计算，以节省内存
            last_hidden_states = bert(input_ids, attention_mask=attention_mask)
        bert_output = last_hidden_states[0][:, 0]
        outputs = net(bert_output)
        for i in range(len(outputs)):
            outputs_list.append(float(outputs[i][0]))
        print(f"Batch {batch} finished.")
    else:
        contents_1 = contents[batch*10:batch*10+2]
        tokens = tokenizer(contents_1, padding=True)
        input_ids = torch.tensor(tokens["input_ids"])
        attention_mask = torch.tensor(tokens["attention_mask"])
        with torch.no_grad():  # 禁用梯度计算，以节省内存
            last_hidden_states = bert(input_ids, attention_mask=attention_mask)
        bert_output = last_hidden_states[0][:, 0]
        outputs = net(bert_output)
        for i in range(len(outputs)):
            outputs_list.append(float(outputs[i][0]))
        print(f"Batch {batch} finished.")

Batch 0 finished.
Batch 1 finished.
Batch 2 finished.
Batch 3 finished.
Batch 4 finished.
Batch 5 finished.
Batch 6 finished.
Batch 7 finished.
Batch 8 finished.
Batch 9 finished.
Batch 10 finished.
Batch 11 finished.
Batch 12 finished.
Batch 13 finished.
Batch 14 finished.
Batch 15 finished.
Batch 16 finished.
Batch 17 finished.
Batch 18 finished.
Batch 19 finished.
Batch 20 finished.
Batch 21 finished.
Batch 22 finished.
Batch 23 finished.
Batch 24 finished.
Batch 25 finished.
Batch 26 finished.
Batch 27 finished.
Batch 28 finished.
Batch 29 finished.
Batch 30 finished.
Batch 31 finished.
Batch 32 finished.
Batch 33 finished.
Batch 34 finished.
Batch 35 finished.
Batch 36 finished.
Batch 37 finished.
Batch 38 finished.
Batch 39 finished.
Batch 40 finished.
Batch 41 finished.
Batch 42 finished.
Batch 43 finished.
Batch 44 finished.
Batch 45 finished.
Batch 46 finished.
Batch 47 finished.
Batch 48 finished.
Batch 49 finished.
Batch 50 finished.
Batch 51 finished.
Batch 52 finished.
Bat

In [42]:
len(outputs_list)

6282

In [43]:
sum(outputs_list) / len(outputs_list)

0.4169330405423033